## Forecasting

We dont use de vaucher infromaction because could add noise to the data.

Use the last two months as testing data.

ARIMA as a reference model.

MAE: for easy interpretability.

In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import matplotlib.pyplot as plt

import sys
sys.path.append('scripts')

import pandas as pd
import numpy as np
import warnings

# Suppress all UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)
%load_ext autoreload
%autoreload 3
from utils import *

c:\Users\kenia\anaconda3\envs\cobre\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Database found at: c:\Users\kenia\OneDrive\Documents\GitHub\cobre\scripts\..\data\olist.sqlite


In [2]:
%reload_ext autoreload

In [3]:
orders_df = load_table('orders')
order_payments_df = load_table('order_payments')

# basic preprocessing
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_df['order_approved_at'] = pd.to_datetime(orders_df['order_approved_at'])
orders_df['order_delivered_carrier_date'] = pd.to_datetime(orders_df['order_delivered_carrier_date'])
orders_df['order_delivered_customer_date'] = pd.to_datetime(orders_df['order_delivered_customer_date'])
orders_df['order_estimated_delivery_date'] = pd.to_datetime(orders_df['order_estimated_delivery_date'])

order_payments_df['payment_type'] = order_payments_df['payment_type'].str.upper().str.replace('_', ' ')

In [4]:
# objective variable: payment_value
df = pd.merge(orders_df, order_payments_df, on='order_id')

for c in df.payment_type.unique():
    aux = df.query(f'payment_type == "{c}"')
    print(f'{c } {"--"*5} ')
    display(aux.loc[:,['payment_sequential','payment_installments','payment_value']].describe().T)

CREDIT CARD ---------- 


,count,mean,std,min,25%,50%,75%,max
payment_sequential,76795.0,1.004167,0.064619,1.00,1.00,1.00,1.00,3.00
payment_installments,76795.0,3.507155,2.850990,0.00,1.00,3.00,5.00,24.00
payment_value,76795.0,163.319021,222.119311,0.01,62.21,106.87,181.21,13664.08


VOUCHER ---------- 


,count,mean,std,min,25%,50%,75%,max
payment_sequential,5775.0,2.602424,2.550195,1.0,1.000,2.00,3.0,29.00
payment_installments,5775.0,1.000000,0.000000,1.0,1.000,1.00,1.0,1.00
payment_value,5775.0,65.703354,115.519185,0.0,18.035,39.28,80.0,3184.34


BOLETO ---------- 


,count,mean,std,min,25%,50%,75%,max
payment_sequential,19784.0,1.000051,0.007110,1.00,1.0000,1.00,1.0000,2.00
payment_installments,19784.0,1.000000,0.000000,1.00,1.0000,1.00,1.0000,1.00
payment_value,19784.0,145.034435,213.581061,11.62,55.5225,93.89,160.7625,7274.88


DEBIT CARD ---------- 


,count,mean,std,min,25%,50%,75%,max
payment_sequential,1529.0,1.034663,0.186527,1.00,1.0,1.0,1.00,3.0
payment_installments,1529.0,1.000000,0.000000,1.00,1.0,1.0,1.00,1.0
payment_value,1529.0,142.570170,245.793401,13.38,51.1,89.3,154.32,4445.5


NOT DEFINED ---------- 


,count,mean,std,min,25%,50%,75%,max
payment_sequential,3.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
payment_installments,3.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
payment_value,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
daily_revenue = df.groupby(df['order_purchase_timestamp'].dt.date)['payment_value'].sum().reset_index()
daily_revenue.columns = ['ds', 'y']  # Prophet requiere estas columnas

print(daily_revenue.ds.min(), daily_revenue.ds.max())
daily_revenue.describe()

2016-09-04 2018-10-17


,y
count,633.000000
mean,25290.477283
std,15123.570267
min,19.620000
25%,15005.060000
50%,23559.520000
75%,34385.080000
max,179200.030000


In [13]:
# Paso 3: Entrenar modelo Prophet
model = Prophet()
model.fit(daily_revenue)

# Paso 4: Crear fechas futuras y predecir
future = model.make_future_dataframe(periods=90)  # 30 días hacia adelante
forecast = model.predict(future)

20:07:09 - cmdstanpy - INFO - Chain [1] start processing
20:07:09 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2016-09-04,4825.279784,-16933.982344,5418.008248,4825.279784,4825.279784,-10411.606319,-10411.606319,-10411.606319,-4305.354439,-4305.354439,-4305.354439,-6106.251881,-6106.251881,-6106.251881,0.0,0.0,0.0,-5586.326536
1,2016-09-05,4867.847142,-9605.857148,12356.102338,4867.847142,4867.847142,-2850.575839,-2850.575839,-2850.575839,3380.218398,3380.218398,3380.218398,-6230.794237,-6230.794237,-6230.794237,0.0,0.0,0.0,2017.271303
2,2016-09-13,5208.386008,-9629.939659,11934.136247,5208.386008,5208.386008,-3721.089965,-3721.089965,-3721.089965,2966.969364,2966.969364,2966.969364,-6688.059329,-6688.059329,-6688.059329,0.0,0.0,0.0,1487.296044
3,2016-10-02,6017.165815,-14015.842175,9001.265316,6017.165815,6017.165815,-8285.573310,-8285.573310,-8285.573310,-4305.354439,-4305.354439,-4305.354439,-3980.218871,-3980.218871,-3980.218871,0.0,0.0,0.0,-2268.407495
4,2016-10-03,6059.733174,-6278.427615,16531.585987,6059.733174,6059.733174,-203.069632,-203.069632,-203.069632,3380.218398,3380.218398,3380.218398,-3583.288030,-3583.288030,-3583.288030,0.0,0.0,0.0,5856.663541


In [15]:
plot_plotly(model, forecast)

In [16]:
plot_components_plotly(model, forecast)